In [4]:
import cvxpy as cp
import numpy as np

In [20]:
import cvxpy as cp
import numpy as np

# Definir variáveis k, f
k, f = 1, 1
# Definir as matrizes A, B e C
A = np.array([[0, 0, 1, 0],
              [0, 0, 0, 1],
              [-k, k, -f, f],
              [k, -k, f, -f]])

B1 = np.array([[0], [0], [0], [1]])
B2 = np.array([[0], [0], [0], [1]])

# --- CORREÇÃO 1 ---
# As matrizes C e D precisam ter 4 linhas para serem compatíveis com as
# variáveis X e W (4x4) na LMI. O código original tinha 3 linhas.
# Adicionei uma linha de zeros, mas verifique se isso está correto
# para o seu problema específico.
C = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]])  # Shape agora é (4, 4)

D = np.array([[0], [0], [1], [0]])  # Shape agora é (4, 1)

n, p = A.shape[0], A.shape[1]

# Definir as variáveis de otimização
X, W = [cp.Variable((n, n), symmetric=True) for i in range(2)]
Z = cp.Variable((1, n))

# Blocos para LMI1
M11 = X
M12 = C @ W + D @ Z
M21 = W @ C.T + Z.T @ D.T
M22 = W

# Blocos para LMI2
N11 = A @ W + B2 @ Z + W @ A.T + Z.T @ B2.T
N12 = B1
N21 = B1.T
# --- CORREÇÃO 2 ---
# A dimensão de N22 deve ser compatível com N21.
# B1.T (N21) tem shape (1,4), então N22 deve ter 1 linha.
# A dimensão de np.eye deve ser o número de colunas de B1.
N22 = -np.eye(B1.shape[1])  # Shape agora é (1, 1)

# Construir as matrizes de blocos LMI
LMI1 = cp.bmat([
    [M11, M12],
    [M21, M22]
])

LMI2 = cp.bmat([
    [N11, N12],
    [N21, N22]
])

rho = cp.Variable()
# Definir a função objetivo: min (rho)
objective = cp.Minimize(rho)

# Definir as restrições
constraints = [cp.trace(X) <= rho,
               LMI1 >> 0,
               LMI2 << 0]

# Criar e resolver o problema de otimização
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCS) # É uma boa prática especificar o solver

# Verificar o status da solução
if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Status:", problem.status, "\n")
    print("Valor mínimo de Tr(X):")
    print(problem.value)
    print("\nMatriz W:")
    print(W.value)
    print("\nMatriz X:")
    print(X.value)
    print("\nVetor Z:")
    print(Z.value)
else:
    print("O problema não tem uma solução ótima. Status:", problem.status)


Status: optimal 

Valor mínimo de Tr(X):
1.2175677861995216

Matriz W:
[[ 1.78378804e-01  1.31671934e-01 -1.69489190e-06 -5.55395901e-02]
 [ 1.31671934e-01  1.40512453e-01  5.55415058e-02 -1.57610620e-06]
 [-1.69489190e-06  5.55415058e-02  1.02244665e-01  4.67015527e-02]
 [-5.55395901e-02 -1.57610620e-06  4.67015527e-02  2.12610608e-01]]

Matriz X:
[[ 1.78389706e-01  1.31667238e-01 -1.48948309e-01 -3.18085031e-17]
 [ 1.31667238e-01  1.40506290e-01 -2.59315440e-01  1.55103831e-17]
 [-1.48948309e-01 -2.59315440e-01  8.98672526e-01 -3.24557424e-16]
 [-3.18085031e-17  1.55103831e-17 -3.24557424e-16 -2.53466442e-07]]

Vetor Z:
[[-0.14894348 -0.25931644 -0.22144497 -0.27855178]]
